In [1]:
### Uncomment next block to download datasets and unzip in ../examples/icbeb/data

In [2]:
#!wget http://hhbucket.oss-cn-hongkong.aliyuncs.com/TrainingSet1.zip
#!wget http://hhbucket.oss-cn-hongkong.aliyuncs.com/TrainingSet2.zip
#!wget http://hhbucket.oss-cn-hongkong.aliyuncs.com/TrainingSet3.zip
#!wget http://2018.icbeb.org/file/REFERENCE.csv

#!unzip TrainingSet1.zip
#!unzip TrainingSet2.zip
#!unzip TrainingSet3.zip

#!mv TrainingSet1/* ../examples/icbeb/data/*
#!mv TrainingSet2/* ../examples/icbeb/data/*
#!mv TrainingSet3/* ../examples/icbeb/data/*
#!mv REFERENCE.csv ../examples/icbeb/data/REFERENCE.csv

#rm -rf TrainingSet1.zip
#rm -rf TrainingSet2.zip
#rm -rf TrainingSet3.zip
#rm -rf TrainingSet1
#rm -rf TrainingSet2
#rm -rf TrainingSet3

In [3]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import
import argparse
import collections
import json
import keras
from keras.layers import Input,Conv1D,MaxPooling1D,Flatten,TimeDistributed,Dense,Add,Activation,Dropout,BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
import math
import matplotlib.pyplot as plt
%matplotlib inline
import network
import numpy as np
import os
import random
import scipy.io as sio
import tensorflow as tf
import time
import tqdm
import scipy.stats as sst
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils.multiclass import unique_labels
import sys
sys.path.append("../../../ecg")

random.seed(7)
tf.set_random_seed(7)
MAX_EPOCHS = 100
maxlength=35*500#pad to 35s, 500Hz
batch=32
n_leads=12

Using TensorFlow backend.


In [4]:
class Preproc:
    def __init__(self, ecg, labels):
        self.mean, self.std = compute_mean_std(ecg)
        self.classes = sorted(set(l for label in labels for l in label))
        self.int_to_class = dict( zip(range(len(self.classes)), self.classes))
        self.class_to_int = {c : i for i, c in self.int_to_class.items()}
    def process(self, x, y):
        return self.process_x(x), self.process_y(y)
    def process_x(self, x):
        x = pad_x(x)
        x = (x - self.mean) / self.std
        return x
    def process_y(self, y):
        y = pad_y([[self.class_to_int[c] for c in s] for s in y], val=9, dtype=np.int32) #for truncate
        y = keras.utils.np_utils.to_categorical(y, num_classes=len(self.classes)+1)
        return y

In [5]:
def pad_x(x, val=0, dtype=np.float32):
    max_len = int(min(maxlength,max(len(i) for i in x))/STEP)*STEP
    padded = np.full((len(x), max_len,12), val, dtype=dtype)
    for e, i in enumerate(x):
        padded[e, :min(len(i),max_len),:] = i[0:min(max_len,i.shape[0]),:]
    return padded

In [6]:
def pad_y(x, val=0, dtype=np.float32):
    max_len = min(int(maxlength/STEP),max(len(i) for i in x))
    padded = np.full((len(x), max_len), val, dtype=dtype)
    for e, i in enumerate(x):
        padded[e, :min(len(i),max_len)] = i[0:min(max_len,len(i))]
    return padded

In [7]:
def compute_mean_std(x):
    x=np.vstack(x)
    return (np.mean(x).astype(np.float32),
           np.std(x).astype(np.float32))

In [8]:
def load_dataset(data_json):
    with open(data_json, 'r') as fid:
        data = [json.loads(l) for l in fid]
    labels = []; ecgs = []
    for d in tqdm.tqdm(data):
        labels.append(d['labels'])
        ecgs.append(load_ecg(d['ecg']))
    return ecgs, labels

In [9]:
def load_ecg(record):
    if os.path.splitext(record)[1] == ".npy":
        ecg = np.load(record)
    elif os.path.splitext(record)[1] == ".mat":
        ecg = sio.loadmat(record)['ECG'][0][0][2].transpose().squeeze()
    else: # Assumes binary 16 bit integers
        with open(record, 'r') as fid:
            ecg = np.fromfile(fid, dtype=np.int16)
    trunc_samp = STEP * int(len(ecg) / STEP)
    return ecg[:trunc_samp]

In [10]:
def load_ecg_mat(ecg_file):
    return sio.loadmat(ecg_file)['ECG'][0][0][2].transpose().squeeze()

In [11]:
def load_all(data_path):
    label_file = os.path.join(data_path, "REFERENCE.csv")
    with open(label_file, 'r') as fid:
        records = [l.strip().split(",") for l in fid]
    dataset = []
    for record, label1,label2,label3 in tqdm.tqdm(records):
        ecg_file = os.path.join(data_path, record + ".mat")
        ecg_file = os.path.abspath(ecg_file)
        if os.path.isfile(ecg_file):
            ecg = load_ecg_mat(ecg_file)
            num_labels = int(ecg.shape[0] / STEP)
            dataset.append((ecg_file, [label1]*num_labels))
    return dataset 

In [12]:
def split(dataset, dev_frac):
    dev_cut = int(dev_frac * len(dataset))
    random.shuffle(dataset)
    dev = dataset[:dev_cut]
    train = dataset[dev_cut:]
    return train, dev

In [13]:
def make_json(save_path, dataset):
    with open(save_path, 'w') as fid:
        for d in dataset:
            datum = {'ecg' : d[0],
                     'labels' : d[1]}
            json.dump(datum, fid)
            fid.write('\n')

In [14]:
def balance_test_set():
    data_path = "../examples/icbeb/data/"
    dataset = load_all(data_path)
    test=[]
    train_lcn=[]
    valid_lcn=[]
    train_hannun=[]
    for j in range(10):
        classj=[]
        for i in range(len(dataset)):
            if int(dataset[i][1][0])==j:
                classj.append(dataset[i])
        np.random.shuffle(classj)
        test=test+classj[0:50]
        valid_lcn=valid_lcn+classj[50:65]
        train_lcn=train_lcn+classj[65:]
        train_hannun=train_hannun+classj[50:]
    np.random.shuffle(test)
    np.random.shuffle(train_lcn)
    np.random.shuffle(train_hannun)
    np.random.shuffle(valid_lcn)
    return train_lcn,valid_lcn,train_hannun,test#same test set, the validation set for LCN model is balanced
    

In [15]:
def build_json():
    train_lcn,valid_lcn,train_hannun,test=balance_test_set()
    make_json("../examples/icbeb/train_hannun.json", train_hannun)
    make_json("../examples/icbeb/train_lcn.json", train_lcn)
    make_json("../examples/icbeb/valid.json", valid_lcn)
    make_json("../examples/icbeb/test.json", test)

In [16]:
def make_save_dir(dirname, experiment_name):
    start_time = str(int(time.time())) + '-' + str(random.randrange(1000))
    save_dir = os.path.join(dirname, experiment_name, start_time)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    return save_dir

In [17]:
def get_filename_for_saving(save_dir):
    return os.path.join(save_dir,
            "{val_loss:.3f}-{val_acc:.3f}-{epoch:03d}-{loss:.3f}-{acc:.3f}.hdf5")

In [18]:
def calculate_architecture(m,n_leads,n_class):
    n=0
    bestc=0
    for c in range(1,m):
        if c*(c*c+1)>m or n_class+n_class*c>m or c*(n_leads*c+1)>m:
                break
            #elif c*(k*c+1)*8+(4+4*c)>n:
                #n=c*(k*c+1)
        elif min(c*(c*c+1),n_class+n_class*c,c*(n_leads*c+1))>n:
            n=min(c*(c*c+1),n_class+n_class*c,c*(n_leads*c+1))#optimizing min layer param/bottle nack        
            bestc=c     
    return bestc

In [19]:
def dup_label(Y,d):
    y=np.zeros((Y.shape[0],Y.shape[1]*d,Y.shape[2]))
    for i in range(Y.shape[1]):
        for j in range(d):
            y[:,d*i+j,:]=Y[:,i,:]
    return y

In [20]:
def train_hannun_model(params):
    print("Loading training set...")
    train = load_dataset(params['train'])
    print("Building preprocessor...")
    preproc = Preproc(*train)
    print("Training size: " + str(len(train[0])) + " examples.")
    
    model = network.build_network(**params)
    stopping = keras.callbacks.EarlyStopping(patience=8)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        factor=0.1,
        patience=2,
        min_lr=params["learning_rate"] * 0.001)
    checkpointer = keras.callbacks.ModelCheckpoint(params['model_dir'],
        save_best_only=True)
    batch_size = params.get("batch_size", 32)
    train_x, train_y = preproc.process(*train)
    X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.1, random_state=42,
                                                        stratify=train_y[:,0,:])
   
    print(model.summary())
    print(X_train.shape)
    #y_train_1=dup_label(y_train,2)
    #y_test_1=dup_label(y_test,2)
    y_train_1=y_train
    y_test_1=y_test
    print(y_train_1.shape)
    history=model.fit(
            X_train, y_train_1,
            batch_size=batch_size,
            epochs=MAX_EPOCHS,
            validation_data=(X_test, y_test_1),
            callbacks=[checkpointer, reduce_lr, stopping])
    return model,history

In [21]:
def LCN(m,params):
    bn=params['bn']
    skip=params['skip']
    repeat=params['repeat']
    n_leads=params['n_leads']
    n_class=params['n_class']
    c=calculate_architecture(m,n_leads,n_class)
    n_maxpool=calculate_maxpool(params)
    print('n_maxpool: '+str(n_maxpool))
    print('c= '+str(c))
   
    inputs=Input(shape=(None,n_leads))
    if not bn:
        layer=Conv1D(c,c,padding='same')(inputs)
    else:
        layer=BatchNormalization(axis=-1)(inputs)
        layer=Conv1D(c,c,padding='same')(layer)
    layer1=layer
    if params['activation']=='relu':
        layer=Activation('relu')(layer)
    elif params['activation']=='leaky':
        layer=LeakyReLU(alpha=0.3)(layer)
        
   
    if bn:
        layer=BatchNormalization(axis=-1)(layer)
        
    layer=MaxPooling1D(2)(layer)
    if skip!=0:
        layer1=MaxPooling1D(2)(layer1)
    
    for i in range(int(n_maxpool)-1):
        for j in range(repeat):
            layer=Conv1D(c,c,padding='same')(layer)
            n_layer=i*repeat+j+2
            if skip!=0 and n_layer%skip==1:    
                layer=Add()([layer,layer1])
                layer1=layer
            if params['activation']=='relu':
                layer=Activation('relu')(layer)
            elif params['activation']=='leaky':
                layer=LeakyReLU(alpha=0.3)(layer)
        
            if bn:
                layer=BatchNormalization(axis=-1)(layer)
        
        layer=MaxPooling1D(2)(layer)
        if skip!=0:
            layer1=MaxPooling1D(2)(layer1)
        
    layer=TimeDistributed(Dense(n_class,activation='softmax'))(layer)#is it the same as sigmoid?
    model=Model(inputs=inputs,outputs=layer)
    model.compile('adam',loss='categorical_crossentropy',metrics=['acc'])
    print(model.summary())
    return model

In [22]:
def train_LCN(params):
    repeat=params['repeat']
    skip=params['skip']
    bn=params['bn']
    batch=params['batch_size']
    n_leads=params['n_leads']
    n_class=params['n_class']
    
    print("Loading training set...")
    train = load_dataset(params['train'])
    valid=load_dataset(params['valid'])
    print("Building preprocessor...")
    preproc = Preproc(*train)
    print("Training size: " + str(len(train[0])) + " examples.")
    train_x, train_y = preproc.process(*train)
    valid_x, valid_y = preproc.process(*valid)
    print(train_x.shape)
    
    stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=8)
    cp = keras.callbacks.ModelCheckpoint(
        params['model_dir'],
        save_best_only=True)
    
    weights=np.zeros((10,))
    weights[0:9]=1.0/(sum(train_y[:,0,0:9])/train_y.shape[0])
    sample_weight=np.matmul(train_y,weights)
    
    y_train_1=dup_label(train_y,4)
    y_test_1=dup_label(valid_y,4)
    
    model=LCN(train_x.shape[0],params)
    
    print(model.summary())
    print(train_x.shape)
    print(train_y.shape)
    
    history=model.fit(
            train_x, train_y,
            batch_size=params['batch_size'],
            epochs=MAX_EPOCHS,
            validation_data=(valid_x, valid_y),
        sample_weight=sample_weight[:,0],
            callbacks=[cp,stopping])
    return model,history

In [23]:
from sklearn.metrics import confusion_matrix
import sys
def evaluate_hannun_model(params):
    model= network.build_network(**params)
    model.load_weights(params['model_dir'])
    data_path = params['test']
    data = load_dataset(data_path)
    print('test size: '+str(len(data[0])))
    train = load_dataset(params['train'])
    preproc = Preproc(*train)
    
    #calculate prior (copied from Hannun's implementation)
    data_path = params['train']
    with open(data_path, 'r') as fid:
        train_labels = [json.loads(l)['labels'] for l in fid]
    counts = collections.Counter(preproc.class_to_int[l[0]] for l in train_labels)
    counts = sorted(counts.most_common(), key=lambda x: x[0])
    counts = zip(*counts)[1]
    smooth = 500
    counts = np.array(counts)[None, None, :]
    total = np.sum(counts) + counts.shape[1]
    prior = (counts + smooth) / float(total)
    
    probs = []
    labels = []
    for x, y  in zip(*data):
        x, y = preproc.process([x], [y])
        probs.append(model.predict(x))
        labels.append(y)   
  
    preds = []
    ground_truth = []
    for p, g in zip(probs, labels):
        preds.append(sst.mode(np.argmax(p[:,:,0:9]/prior, axis=2).squeeze())[0][0])#1 for no truncate
        ground_truth.append(sst.mode(np.argmax(g, axis=2).squeeze())[0][0])#1 for no truncate
        
    import sklearn.metrics as skm
    report = skm.classification_report(
            ground_truth, preds,
            target_names=['N','AF','I-AVB','LBBB','RBBB','PAC','PVC','STD','STE'],
            digits=3)
    scores = skm.precision_recall_fscore_support(
                    ground_truth,
                    preds,
                   average=None)
   
    print(report)
    print("CINC Average {:3f}".format(np.mean(scores[2][:3])))
    print(confusion_matrix(ground_truth,preds))

In [24]:
from sklearn.metrics import confusion_matrix
import sys
def evaluate_LCN(params):
    
    repeat=params['repeat']
    skip=params['skip']
    bn=params['bn']
    data_path = params['test']
    data = load_dataset(data_path)
    print('test size: '+str(len(data[0])))
    train = load_dataset(params['train'])

    model=LCN(len(train[0]),params)
    model.load_weights(params['model_dir'])
   
    preproc = Preproc(*train)
    
    probs = []
    labels = []
    for x, y  in zip(*data):
        x, y = preproc.process([x], [y])
        probs.append(model.predict(x))
        labels.append(y)   
    preds = []
    ground_truth = []
    for p, g in zip(probs, labels):
        preds.append(sst.mode(np.argmax(p[:,:,0:9], axis=2).squeeze())[0][0])#1 for no truncate
        ground_truth.append(sst.mode(np.argmax(g, axis=2).squeeze())[0][0])#1 for no truncate   
    import sklearn.metrics as skm
    report = skm.classification_report(
            ground_truth, preds,
      target_names=['N','AF','I-AVB','LBBB','RBBB','PAC','PVC','STD','STE'],
            digits=3)
    scores = skm.precision_recall_fscore_support(
                    ground_truth,
                    preds,
                    average=None)
    print('no prior:')
    print(report)
   
    print(confusion_matrix(ground_truth,preds))


In [25]:
def calculate_maxpool(params):
    tau=params['tau']
    fs=params['fs']
    p=params['p']
    
    return math.ceil(math.log(fs*tau,p))

In [26]:
def autonet(params):   
    params.update({
        'repeat':1,
        'bn':False,
        'skip':0    
    })
    model,history=train_LCN(params)#print total epoch and speed, only return val_loss and train_loss
    train_loss=min(history.history['loss'])
    val_loss=min(history.history['val_loss'])

    min_train_loss=train_loss
    min_val_loss=val_loss
    
    while val_loss<=min_val_loss or train_loss<=min_train_loss:
        if val_loss<=min_val_loss:
            best_model=model
            best_params=params
       
        params['repeat']=params['repeat']+1
        model,history=train_LCN(params)
        train_loss=min(history.history['loss'])
        val_loss=min(history.history['val_loss'])
        
    params['skip']=int(calculate_maxpool(params))-1
    model,history=train_LCN(params)
    train_loss=min(history.history['loss'])
    val_loss=min(history.history['val_loss'])
    
    while val_loss<=min_val_loss or train_loss<=min_train_loss:
        if val_loss<=min_val_loss:
            best_model=model
            best_params=params
        
        params['repeat']=params['repeat']+1
        model,history=train_LCN(params)
        train_loss=min(history.history['loss'])
        val_loss=min(history.history['val_loss'])
        
    params['bn']=True
    model,history=train_LCN(params)
    train_loss=min(history.history['loss'])
    val_loss=min(history.history['val_loss'])
    
    while val_loss<=min_val_loss or train_los<=min_train_loss:
        if val_loss<=min_val_loss:
            best_model=model
            best_params=params
            
        params['repeat']=params['repeat']+1
        model,history=train_LCN(params)
        train_loss=min(history.history['loss'])
        val_loss=min(history.history['val_loss'])
        
       
    return best_model,best_params
            

In [27]:
#run build_json() to create the training, validation, and test sets
#run build_json() only once then comment out this block 
#WARNING: restart the kernel after running build_json() and before running the subsequent codes, or the computer may freeze due to memory limit reached

#build_json()

In [28]:
#due to memory constraints, every time 
#train_hannun_model(params), train_LCN(params), evaluate_hannun_model(params), or evaluate_LCN(params) is called, 
#the kernel needs to be restarted,
#thus the autonet(params) cannot be directly called and one needs to adujst the hyperparameters mannually accoring to the autonet(params) algorithm

method='LCN'#choose from 'Hannun' (benchmark model) or 'LCN'

#hyperparameters to be adjusted manually according to autonet(params)
repeat=4
skip=8
bn=True



train=False #train=False to predict, train=True to train

if method=='Hannun':
    STEP=256
    params={
            "conv_subsample_lengths": [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2],
            "conv_filter_length": 16,
            "conv_num_filters_start": 32,
            "conv_init": "he_normal",
            "conv_activation": "relu",
            "conv_dropout": 0.2,
            "conv_num_skip": 2,
            "conv_increase_channels_at": 4,
            "learning_rate": 0.001,
            "batch_size": 32,
            "generator": True,
            "save_dir": "saved",
            'input_shape':[None,n_leads],#variable length
            'num_categories':10,#9 classes and 1 indicator for noise/0 padding
            'train':'../examples/icbeb/train_ng.json',
            'valid':'../examples/icbeb/valid.json',
            'test':'../examples/icbeb/test.json',
            'model_dir':'../examples/icbeb/hannun_weights.json'
        }
    
    if train:
        model,history=train_hannun_model(params)
            
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.legend(['train loss','val_loss'])
        plt.title('Training and Validation loss on ICBEB')
        plt.show()

        print('total epoch: '+str(len(history.history['loss'])))
        print('min val_loss: %.4f'%min(history.history['val_loss']))
        print('min train loss: %.4f'%min(history.history['loss']))
    else:
         evaluate_hannun_model(params)
        
elif method=='LCN':
    STEP=512
    params={"batch_size": batch,
                'input_shape':[None,n_leads],#variable length
                'n_class':10,#9 classes and 1 indicator for noise/0 padding,
                'repeat':repeat,
                'bn':bn,
                'skip':skip,
                'n_leads':n_leads,
                'train':'../examples/icbeb/train_lcn.json',
                'valid':'../examples/icbeb/valid.json',
                'test':'../examples/icbeb/test.json',
                'model_dir':'../examples/icbeb/lcn_weights.json',
                'p':2,#pooling region, recommend keep the default value of 2
                'fs':500,#sampling frequency in Hz
                'tau':1,#timescale hyperparameter for periodic signals, roughly equal to the period (seconds) of the signal
            'activation':'relu'#choose from 'relu' or 'leaky'
    }
    if train:
        model,history=train_LCN(params)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.legend(['train loss','val_loss'])
        plt.title('Training and Validation loss on ICBEB')
        plt.show()

        print('total epoch: '+str(len(history.history['loss'])))
        print('min val_loss: %.4f'%min(history.history['val_loss']))
        print('min train loss: %.4f'%min(history.history['loss']))
    
    else:
        evaluate_LCN(params)

  0%|          | 26/6292 [00:00<00:24, 255.25it/s]

test size: 450


100%|██████████| 6292/6292 [00:26<00:00, 240.83it/s]


n_maxpool: 9.0
c= 18
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 12)     0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, 12)     48          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 18)     3906        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, 18)     0           conv1d_1[0][0]                   
________________________________________________________________________________________

no prior:
              precision    recall  f1-score   support

           N      0.731     0.760     0.745        50
          AF      0.780     0.920     0.844        50
       I-AVB      0.974     0.740     0.841        50
        LBBB      0.920     0.920     0.920        50
        RBBB      0.930     0.800     0.860        50
         PAC      0.630     0.920     0.748        50
         PVC      0.932     0.820     0.872        50
         STD      0.848     0.560     0.675        50
         STE      0.741     0.860     0.796        50

   micro avg      0.811     0.811     0.811       450
   macro avg      0.832     0.811     0.811       450
weighted avg      0.832     0.811     0.811       450

[[38  0  0  0  0  1  0  4  7]
 [ 0 46  0  2  0  2  0  0  0]
 [ 3  0 37  1  1  4  1  0  3]
 [ 0  1  0 46  0  2  0  0  1]
 [ 1  7  0  0 40  2  0  0  0]
 [ 1  2  0  0  1 46  0  0  0]
 [ 0  2  0  1  0  6 41  0  0]
 [ 6  0  1  0  0  9  2 28  4]
 [ 3  1  0  0  1  1  0  1 43]]
